In [3]:
from glob import glob
import pickle
import os
import numpy as np
from typing import Any, Dict, List, Tuple, Union
#from argoverse.map_representation.map_api import ArgoverseMap
import torch
from torch import nn, optim, utils
from torch.utils.data import IterableDataset, DataLoader, Dataset
from tqdm import tqdm
import sys
import pandas as pd
import dill

In [7]:
!pip install orjson

     |████████████████████████████████| 247 kB 15.7 MB/s            


In [8]:
from environment import Environment, Scene, Node
from utils import maybe_makedirs
from environment import derivative_of

In [9]:
data_columns = pd.MultiIndex.from_product([['position', 'velocity', 'acceleration'], ['x', 'y']])
data_columns

MultiIndex([(    'position', 'x'),
            (    'position', 'y'),
            (    'velocity', 'x'),
            (    'velocity', 'y'),
            ('acceleration', 'x'),
            ('acceleration', 'y')],
           )

In [12]:
desired_max_time = 100
pred_indices = [2, 3]
state_dim = 6
frame_diff = 10
desired_frame_diff = 1
dt = 0.1

standardization = {
    'PEDESTRIAN': {
        'position': {
            'x': {'mean': 0, 'std': 1},
            'y': {'mean': 0, 'std': 1}
        },
        'velocity': {
            'x': {'mean': 0, 'std': 2},
            'y': {'mean': 0, 'std': 2}
        },
        'acceleration': {
            'x': {'mean': 0, 'std': 1},
            'y': {'mean': 0, 'std': 1}
        }
    },
    'VEHICLE': {
        'position': {
            'x': {'mean': 0, 'std': 80},
            'y': {'mean': 0, 'std': 80}
        },
        'velocity': {
            'x': {'mean': 0, 'std': 15},
            'y': {'mean': 0, 'std': 15},
            'norm': {'mean': 0, 'std': 15}
        },
        'acceleration': {
            'x': {'mean': 0, 'std': 4},
            'y': {'mean': 0, 'std': 4},
            'norm': {'mean': 0, 'std': 4}
        },
        'heading': {
            'x': {'mean': 0, 'std': 1},
            'y': {'mean': 0, 'std': 1},
            '°': {'mean': 0, 'std': np.pi},
            'd°': {'mean': 0, 'std': 1}
        }
    }
}
    

In [ ]:
env = Environment(node_type_list=['PEDESTRIAN'], standardization=standardization)
attention_radius = dict()
attention_radius[(env.NodeType.PEDESTRIAN, env.NodeType.PEDESTRIAN)] = 3.0
env.attention_radius = attention_radius

In [ ]:
desired_source = 'eth'
data_class = 'val'

In [ ]:
pa = os.path.join('../experiments/pedestrians/raw', desired_source, data_class)
for subdir, dirs, files in os.walk(pa):
    
    for file in files:
        if file.endswith('.txt'):
            input_data_dict = dict()
            full_data_path = os.path.join(subdir, file)
            print('At', full_data_path)
            data = pd.read_csv(full_data_path, sep='\t', index_col=False, header=None)
            data.columns = ['frame_id', 'track_id', 'pos_x', 'pos_y']
            data['frame_id'] = pd.to_numeric(data['frame_id'], downcast='integer')
            data['track_id'] = pd.to_numeric(data['track_id'], downcast='integer')

            data['frame_id'] = data['frame_id'] // 10

            data['frame_id'] -= data['frame_id'].min()

            data['node_type'] = 'PEDESTRIAN'
            data['node_id'] = data['track_id'].astype(str)
            data.sort_values('frame_id', inplace=True)

            break

# Argoverse

In [19]:
! ls ../../argoverse

 agents_train.npy		    forecasting_test_v1.1.tar.gz
 agents_train_rotation.npy	    forecasting_test_v1.1.tar.gz.1
 agents_train_small.npy		    forecasting_train_v1.1.tar.gz.2
 agents_train_small_transi.npy	   'preprocessing for cvae.ipynb'
 agents_train_transi.npy	    preprocessing.ipynb
 agents_train_transi_rotation.npy   test
 agents_val.npy			    test_new
 agents_val_rotation.npy	    test_obs
 agents_val_transi.npy		    train
 agents_val_transi_rotation.npy     train_original
 argoverse-api.git		    val
 features			    val_original


In [17]:
pkl_list = glob(os.path.join('../../argoverse/val', '*'))
np.random.shuffle(pkl_list)

In [13]:
env = Environment(node_type_list=['VEHICLE'], standardization=standardization)
attention_radius = dict()
attention_radius[(env.NodeType.PEDESTRIAN, env.NodeType.PEDESTRIAN)] = 30.0
env.attention_radius = attention_radius
#env.robot_type = env.NodeType.VEHICLE
scenes = []


In [43]:
def make_env_argo(pkl_list):
    env = Environment(node_type_list=['VEHICLE'], standardization=standardization)
    attention_radius = dict()
    attention_radius[(env.NodeType.VEHICLE, env.NodeType.VEHICLE)] = 30.0
    env.attention_radius = attention_radius
    #env.robot_type = env.NodeType.VEHICLE
    scenes = []


    dfcolumns = ['frame_id', 'track_id', 'pos_x', 'pos_y', 'node_type', 'node_id','agent','scene_idx']
    counter = 0
    for pkl_path in tqdm(pkl_list):
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)    
        agent = (data['track_id0'][0] == data['agent_id']).nonzero()
        agent = int(agent[0])

        scene = data['scene_idx'][0]
        cars = int(data['car_mask'][0].sum())
        cardfs = []

        for i in range(cars):
            #obj = "AGENT" if (i==agent) else "OTHERS"
            isagent = 1 if (i==agent) else 0
            trackid = data['track_id0'][0][i][-6:]
            nodeid = trackid
            in_x = data['pos_2s'][0][i,:,0]
            in_y = data['pos_2s'][0][i,:,1]
            out_x = [data['pos'+str(j)][0][i,0] for j in range(31)]
            out_y = [data['pos'+str(j)][0][i,1] for j in range(31)]
            dic = {'frame_id': list(range(49)), 
                   'track_id':[trackid]*49, 
                   'pos_x': np.concatenate([in_x,out_x],axis=0).tolist(),
                   'pos_y': np.concatenate([in_y,out_y],axis=0).tolist(),
                   'agent':[isagent]*49,
                  }

            thiscar = pd.DataFrame.from_dict(dic)
            cardfs.append(thiscar)

        data = pd.concat(cardfs, ignore_index=True)
        data['node_type']='VEHICLE' #'vehicle'
        data['scene_idx'] = scene

        data['track_id'] = pd.to_numeric(data['track_id'], downcast='integer')
        data['node_id'] = data['track_id'].astype(str)
        data.sort_values('frame_id', inplace=True)



        # Mean Position
        data['pos_x'] = data['pos_x'] - data['pos_x'].mean()
        data['pos_y'] = data['pos_y'] - data['pos_y'].mean()

        max_timesteps = data['frame_id'].max()

        scene = Scene(timesteps=max_timesteps+1, dt=dt, name=desired_source + "_" + data_class, aug_func=None) # aug_func=augment if data_class == 'train' else

        for node_id in pd.unique(data['node_id']):

            node_df = data[data['node_id'] == node_id]
            assert np.all(np.diff(node_df['frame_id']) == 1)

            node_values = node_df[['pos_x', 'pos_y']].values

            if node_values.shape[0] < 2:
                continue

            new_first_idx = node_df['frame_id'].iloc[0]

            x = node_values[:, 0]
            y = node_values[:, 1]
            vx = derivative_of(x, scene.dt)
            vy = derivative_of(y, scene.dt)
            ax = derivative_of(vx, scene.dt)
            ay = derivative_of(vy, scene.dt)

            data_dict = {('position', 'x'): x,
                         ('position', 'y'): y,
                         ('velocity', 'x'): vx,
                         ('velocity', 'y'): vy,
                         ('acceleration', 'x'): ax,
                         ('acceleration', 'y'): ay}

            node_data = pd.DataFrame(data_dict, columns=data_columns)
            node = Node(node_type=env.NodeType.PEDESTRIAN, node_id=node_id, data=node_data)
            node.first_timestep = new_first_idx

            scene.nodes.append(node)

        scenes.append(scene)
        counter += 1
        #     if counter > 100:
        #         break
        #print(f'Processed {len(scenes):.2f} scene for data class {data_class}')

    env.scenes = scenes
    return env


In [44]:
pkl_list = glob(os.path.join('../../argoverse/val', '*'))
np.random.shuffle(pkl_list)
env_val = make_env_argo(pkl_list)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39472/39472 [23:14<00:00, 28.31it/s]


In [46]:
! mkdir ../experiments/processed

In [50]:
len(env_val.scenes)

39472

In [51]:
small_val = env_val.scenes[:5000]
env_val.scenes = small_val

In [52]:
data_dict_path = os.path.join('../experiments/processed', '_'.join(['argoverse', 'val_small']) + '.pkl')
if len(scenes) > 0:
    with open(data_dict_path, 'wb') as f:
        dill.dump(env_val, f, protocol=dill.HIGHEST_PROTOCOL)


In [48]:
pkl_list = glob(os.path.join('../../argoverse/train', '*'))
np.random.shuffle(pkl_list)
env_train = make_env_argo(pkl_list)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 205942/205942 [2:10:35<00:00, 26.28it/s]


In [ ]:
len(env_train.scenes)

In [49]:
data_dict_path = os.path.join('../experiments/processed', '_'.join(['argoverse', 'train']) + '.pkl')
if len(scenes) > 0:
    with open(data_dict_path, 'wb') as f:
        dill.dump(env_train, f, protocol=dill.HIGHEST_PROTOCOL)

# Pedestrian

val and val_small are reversed

In [67]:
pkl_list = glob(os.path.join('../../pedestrian/processed/val', '*'))
np.random.shuffle(pkl_list)
env_val = make_env_ped(pkl_list)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22937/22937 [02:23<00:00, 159.48it/s]


In [68]:
data_dict_path = os.path.join('../experiments/processed', '_'.join(['ped', 'val_small']) + '.pkl')
if len(scenes) > 0:
    with open(data_dict_path, 'wb') as f:
        dill.dump(env_val, f, protocol=dill.HIGHEST_PROTOCOL)


In [69]:
small_val = env_val.scenes[:5000]
env_val.scenes = small_val

In [70]:
data_dict_path = os.path.join('../experiments/processed', '_'.join(['ped', 'val']) + '.pkl')
if len(scenes) > 0:
    with open(data_dict_path, 'wb') as f:
        dill.dump(env_val, f, protocol=dill.HIGHEST_PROTOCOL)


In [71]:
pkl_list = glob(os.path.join('../../pedestrian/processed/train', '*'))
np.random.shuffle(pkl_list)
env_train = make_env_ped(pkl_list)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 217953/217953 [24:57<00:00, 145.57it/s]


In [72]:
data_dict_path = os.path.join('../experiments/processed', '_'.join(['ped', 'train']) + '.pkl')
if len(scenes) > 0:
    with open(data_dict_path, 'wb') as f:
        dill.dump(env_train, f, protocol=dill.HIGHEST_PROTOCOL)

In [56]:
pkl_list = pkl_list[:1]

In [55]:
data.keys()

dict_keys(['man_mask', 'pos_enc', 'vel_enc', 'pos0', 'vel0', 'pos1', 'vel1', 'pos2', 'vel2', 'pos3', 'vel3', 'pos4', 'vel4', 'pos5', 'vel5', 'pos6', 'vel6', 'pos7', 'vel7', 'pos8', 'vel8', 'pos9', 'vel9', 'pos10', 'vel10', 'pos11', 'vel11', 'pos12', 'vel12', 'scene_idx'])

In [65]:
def make_env_ped(pkl_list):
    env = Environment(node_type_list=['PEDESTRIAN'], standardization=standardization)
    attention_radius = dict()
    attention_radius[(env.NodeType.PEDESTRIAN, env.NodeType.PEDESTRIAN)] = 30.0
    env.attention_radius = attention_radius
    #env.robot_type = env.NodeType.VEHICLE
    scenes = []


    dfcolumns = ['frame_id', 'track_id', 'pos_x', 'pos_y', 'node_type', 'node_id','agent','scene_idx']
    counter = 0
    for pkl_path in tqdm(pkl_list):
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)    

        scene = data['scene_idx'][0]
        cars = int(data['man_mask'][0].sum())
        cardfs = []

        for i in range(cars):
            #obj = "AGENT" if (i==agent) else "OTHERS"
            isagent = 1 if (i==0) else 0
            trackid = i
            nodeid = trackid
            in_x = data['pos_enc'][i,:,0]
            in_y = data['pos_enc'][i,:,1]
            out_x = [data['pos'+str(j)][i,0] for j in range(13)]
            out_y = [data['pos'+str(j)][i,1] for j in range(13)]
            dic = {'frame_id': list(range(20)), 
                   'track_id':[trackid]*20, 
                   'pos_x': np.concatenate([in_x,out_x],axis=0).tolist(),
                   'pos_y': np.concatenate([in_y,out_y],axis=0).tolist(),
                   'agent':[isagent]*20,
                  }

            thiscar = pd.DataFrame.from_dict(dic)
            cardfs.append(thiscar)

        data = pd.concat(cardfs, ignore_index=True)
        data['node_type']='PEDESTRIAN' #'vehicle'
        data['scene_idx'] = scene

        data['track_id'] = pd.to_numeric(data['track_id'], downcast='integer')
        data['node_id'] = data['track_id'].astype(str)
        data.sort_values('frame_id', inplace=True)


        # Mean Position
        data['pos_x'] = data['pos_x'] - data['pos_x'].mean()
        data['pos_y'] = data['pos_y'] - data['pos_y'].mean()

        max_timesteps = data['frame_id'].max()

        scene = Scene(timesteps=max_timesteps+1, dt=dt, name=desired_source + "_" + data_class, aug_func=None) # aug_func=augment if data_class == 'train' else

        for node_id in pd.unique(data['node_id']):

            node_df = data[data['node_id'] == node_id]
            assert np.all(np.diff(node_df['frame_id']) == 1)

            node_values = node_df[['pos_x', 'pos_y']].values

            if node_values.shape[0] < 2:
                continue

            new_first_idx = node_df['frame_id'].iloc[0]

            x = node_values[:, 0]
            y = node_values[:, 1]
            vx = derivative_of(x, scene.dt)
            vy = derivative_of(y, scene.dt)
            ax = derivative_of(vx, scene.dt)
            ay = derivative_of(vy, scene.dt)

            data_dict = {('position', 'x'): x,
                         ('position', 'y'): y,
                         ('velocity', 'x'): vx,
                         ('velocity', 'y'): vy,
                         ('acceleration', 'x'): ax,
                         ('acceleration', 'y'): ay}

            node_data = pd.DataFrame(data_dict, columns=data_columns)
            node = Node(node_type=env.NodeType.PEDESTRIAN, node_id=node_id, data=node_data)
            node.first_timestep = new_first_idx

            scene.nodes.append(node)

        scenes.append(scene)
        counter += 1
        #     if counter > 100:
        #         break
        #print(f'Processed {len(scenes):.2f} scene for data class {data_class}')

    env.scenes = scenes
    return env


In [5]:
import torch
from torch import nn, optim, utils
import numpy as np
import os
import time
import dill
import json
import random
import pathlib
import warnings
from tqdm import tqdm
import visualization
import evaluation
import matplotlib.pyplot as plt
from argument_parser import args
from model.model_registrar import ModelRegistrar
from model.model_utils import cyclical_lr
from model.dataset import EnvironmentDataset, collate
from tensorboardX import SummaryWriter



usage: ipykernel_launcher.py [-h] [--conf CONF] [--debug]
                             [--preprocess_workers PREPROCESS_WORKERS]
                             [--offline_scene_graph OFFLINE_SCENE_GRAPH]
                             [--dynamic_edges DYNAMIC_EDGES]
                             [--edge_state_combine_method EDGE_STATE_COMBINE_METHOD]
                             [--edge_influence_combine_method EDGE_INFLUENCE_COMBINE_METHOD]
                             [--edge_addition_filter EDGE_ADDITION_FILTER [EDGE_ADDITION_FILTER ...]]
                             [--edge_removal_filter EDGE_REMOVAL_FILTER [EDGE_REMOVAL_FILTER ...]]
                             [--override_attention_radius OVERRIDE_ATTENTION_RADIUS]
                             [--incl_robot_node] [--map_encoding] [--augment]
                             [--node_freq_mult_train] [--node_freq_mult_eval]
                             [--scene_freq_mult_train]
                             [--scene_freq_mult_eval] [--scene

SystemExit: 2

In [9]:
from model.trajectron import Trajectron
from model.model_registrar import ModelRegistrar
import json 

log_writer = None

model_registrar = ModelRegistrar('', 'cuda:0')
conf ='../experiments/pedestrians/models/argo/config.json'
with open(conf, 'r', encoding='utf-8') as conf_json:
    hyperparams = json.load(conf_json)



In [ ]:
model_dir = os.path.join(args.log_dir, 'int_ee')
model_registrar = ModelRegistrar(model_dir, args.eval_device)
model_registrar.load_models(iter_num=12)


In [19]:
model_registrar.model_dir = '../experiments/argoverse/models/models_15_Jan_2022_21_03_31_argo_first'
model_registrar.load_models(12)


Loading from ../experiments/argoverse/models/models_15_Jan_2022_21_03_31_argo_first/model_registrar-12.pt
Loaded!



In [22]:
pytorch_total_params = sum(p.numel() for p in model_registrar.parameters())
pytorch_total_params

127654

In [17]:
trajectron = Trajectron(model_registrar,
                        hyperparams,
                        log_writer,
                        'cuda:0')

In [18]:
trajectron.node_models_dict

{}